PyTorch Dataset
Time to refresh your PyTorch Datasets knowledge!

Before model training can commence, you need to load the data and pass it to the model in the right format. In PyTorch, this is handled by Datasets and DataLoaders. Let's start with building a PyTorch Dataset for our water potability data.

In this exercise, you will define a class called WaterDataset to load the data from a CSV file. To do this, you will need to implement the three methods which PyTorch expects a Dataset to have:

.__init__() to load the data,
.__len__() to return data size,
.__getitem()__ to extract features and label for a single sample.

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load the data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert the DataFrame to numpy array and assingn it to self.data
        self.data = df.to_numpy()
        
# Implement the .__len__() method to return the number of data samples.

    def __len__(self):
        return len(self.data)
    
# In the .__getitem__() method, get the label by slicing self.data to extract its last column for the index idx, similarly to how it's done for the features.

    def __getitem__(self, idx):
        features = self.data[idx, :-1]
        label = self.data[idx, -1]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

In [2]:
# Use the WaterDataset class to load the data.

# Create an instance of the WaterDataset
dataset = WaterDataset('/data/raw_data/water_potability/water_train.csv')

# Get the length of the dataset
print(len(dataset))

# Get the first sample from the dataset
first_sample = dataset[0]
print(first_sample)

1508
(tensor([0.4836, 0.6156, 0.5140, 0.7774, 0.3546, 0.3353, 0.3673, 0.5141, 0.6173]), tensor(1.))


The next step in preparing the training data is to set up a DataLoader. A PyTorch DataLoader can be created from a Dataset to load data, split it into batches, and perform transformations on the data if desired. Then, it yields a data sample ready for training.

In this exercise, you will build a DataLoader based on the WaterDataset. The DataLoader class you will need has already been imported for you from torch.utils.data. Let's get to it!

Instrucciones
100 XP
Create an instance of WaterDataset from water_train.csv, assigning it to dataset_train.
Create dataloader_train based on dataset_train, using a batch size of two and shuffling the samples.
Get a batch of features and labels from the DataLoader and print them.

In [3]:
from torch.utils.data import DataLoader

# Create an instance of the WaterDataset
dataset_train = WaterDataset('water_train.csv')

# Create a DataLoader for the dataset
dataloader_train = DataLoader(
    dataset_train, 
    batch_size=2, 
    shuffle=True
)

# Get a batch of data
features, labels = next(iter(dataloader_train))
print(features, labels)

tensor([[0.6134, 0.5218, 0.2622, 0.6169, 0.6471, 0.4344, 0.6310, 0.5719, 0.6995],
        [0.5452, 0.6216, 0.3414, 0.4914, 0.4572, 0.4211, 0.5222, 0.5734, 0.4462]]) tensor([0., 1.])


PyTorch Model
You will use the OOP approach to define the model architecture. Recall that this requires setting up a model class and defining two methods inside it:

.__init__(), in which you define the layers you want to use;

forward(), in which you define what happens to the model inputs once it receives them; this is where you pass inputs through pre-defined layers.

Let's build a model with three linear layers and ReLU activations. After the last linear layer, you need a sigmoid activation instead, which is well-suited for binary classification tasks like our water potability prediction problem. Here's the model defined using nn.Sequential(), which you may be more familiar with:

net = nn.Sequential(
  nn.Linear(9, 16),
  nn.ReLU(),
  nn.Linear(16, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid(),
)
Let's rewrite this model as a class!

In the .__init__() method, define the three linear layers with dimensions corresponding to the model definition provided and assign them to self.fc1, self.fc2, and self.fc3, respectively.
In the forward() method, pass the model input x through all the layers, remembering to add activations on top of them, similarly how it's already done for the first layer.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        # Pass x through linear layers adding activations
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

In [5]:
# Create an instance of the Net class
model = Net()

# Print the model architecture
print(model)

# Get the model parameters
print(dict(model.named_parameters()))

# Get the model parameters and their shapes
for param in model.parameters():
    print(param.shape)

Net(
  (fc1): Linear(in_features=9, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
)
{'fc1.weight': Parameter containing:
tensor([[ 0.1922, -0.0217,  0.0400, -0.1087, -0.0751, -0.1268,  0.3267, -0.0669,
         -0.3015],
        [-0.2916,  0.2606, -0.2145, -0.1060,  0.1204, -0.0006, -0.0482,  0.1936,
         -0.1853],
        [-0.0603,  0.0081,  0.3327,  0.0040, -0.2450, -0.0972, -0.0151, -0.0382,
          0.0849],
        [-0.2542,  0.3188, -0.3275,  0.1676,  0.1902, -0.1359, -0.2198, -0.0926,
         -0.0491],
        [-0.3257, -0.0137, -0.2639,  0.2183,  0.2283,  0.1144,  0.1347,  0.3025,
         -0.1006],
        [-0.2558, -0.1875, -0.2830, -0.0660, -0.0623,  0.1309, -0.2063, -0.3026,
          0.1782],
        [ 0.1301,  0.0056, -0.1785,  0.1608, -0.2156,  0.3099, -0.0148,  0.1713,
          0.2356],
        [-0.2412,  0.0477,  0.0822,  0.0411, -0.1899,  0.0455, -0.0209,  0.0130,

Optimizers
It's time to explore the different optimizers that you can use for training your model.

A custom function called train_model(optimizer, net, num_epochs) has been defined for you. It takes the optimizer, the model, and the number of epochs as inputs, runs the training loops, and prints the training loss at the end.

Let's use train_model() to run a few short trainings with different optimizers and compare the results!

Define the optimizer as Stochastic Gradient Descent.

In [ ]:
import torch.optim as optim

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
train_model(optimizer, model, num_epochs=5)

Define the optimizer as Root Mean Square Propagation (RMSprop), passing the model's parameters as its first argument.

In [ ]:
# Define the optimizer
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

Define the optimizer as Adaptive Moments Estimation (Adam), setting the learning rate to 0.001.

In [ ]:
# Define the optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001)

Model evaluation
With the training loop sorted out, you have trained the model for 1000 epochs, and it is available to you as net. You have also set up a test_dataloader in exactly the same way as you did with train_dataloader before—just reading the data from the test rather than the train directory.

You can now evaluate the model on test data. To do this, you will need to write the evaluation loop to iterate over the batches of test data, get the model's predictions for each batch, and calculate the accuracy score for it. Let's do it!

Instrucciones
100 XP
Set up the evaluation metric as Accuracy for binary classification and assign it to acc.
For each batch of test data, get the model's outputs and assign them to outputs.
After the loop, compute the total test accuracy and assign it to test_accuracy.

In [ ]:
import torch
from torchmetrics import Accuracy

# Set up binary accuracy metric
acc = Accuracy("binary")

net.eval()
with torch.no_grad():
    for features, labels in dataloader_test:
        # Get predicted probabilities for test data batch
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc.update(preds, labels.view(-1, 1))

# Compute total test accuracy
test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

Initialization and activation
The problems of unstable (vanishing or exploding) gradients are a challenge that often arises in training deep neural networks. In this and the following exercises, you will expand the model architecture that you built for the water potability classification task to make it more immune to those problems.

As a first step, you'll improve the weights initialization by using He (Kaiming) initialization strategy. To do so, you will need to call the proper initializer from the torch.nn.init module, which has been imported for you as init. Next, you will update the activations functions from the default ReLU to the often better ELU.

Call the He (Kaiming) initializer on the weight attribute of the second layer, fc2, similarly to how it's done for fc1.
Call the He (Kaiming) initializer on the weight attribute of the third layer, fc3, accounting for the different activation function used in the final layer.
Update the activation functions in the forward() method from relu to elu.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

        # Apply He initialization
        init.kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        init.kaiming_uniform_(self.fc2.weight, nonlinearity='relu')
        init.kaiming_uniform_(self.fc3.weight, nonlinearity='sigmoid')

    def forward(self, x):
        # Update ReLU activation to ELU
        x = nn.functional.elu(self.fc1(x))
        x = nn.functional.elu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

Batch Normalization
As a final improvement to the model architecture, let's add the batch normalization layer after each of the two linear layers. The batch norm trick tends to accelerate training convergence and protects the model from vanishing and exploding gradients issues.

Both torch.nn and torch.nn.init have already been imported for you as nn and init, respectively. Once you implement the change in the model architecture, be ready to answer a short question on how batch normalization works!

Instrucciones 1/3
35 XP
1
2
3
Add two BatchNorm1d layers assigning them to self.bn1 and self.bn2.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)

        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")

In the forward() method, pass x through the second set of layers: the linear layer, the batch norm layer, and the activations, similarly to how it's done for the first set of layers.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)

        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.elu(x)

        # Pass x through the second set of layers
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.elu(x)

        x = nn.functional.sigmoid(self.fc3(x))
        return x